**OBJETIVO:** Este kernel tem como objetivo realizar ajuste fino utilizando os modelos pré-treinados BERT Multilingual e BERT Portuguese para a tarefa de classificação de textos com ideação suicida.



In [ ]:
# Instalação das bibliotecas 
!pip install scikit-learn
!pip install transformers
!pip install simpletransformers


In [2]:
# Importação das bibliotecas
 
import pandas as pd
from sklearn.model_selection import train_test_split
from simpletransformers.classification import ClassificationModel
from sklearn.metrics import accuracy_score as acc
from sklearn.metrics import precision_score as pr
from sklearn.metrics import recall_score as rec
from sklearn.metrics import f1_score as f1
from sklearn.metrics import roc_auc_score as auc 

**1. PRIMEIRO CICLO DE MODELAGEM**

In [3]:
# Leitura dos arquivos de textos com e sem ideação suicida
suicide_data = pd.read_csv('suicidas.txt',sep='delimiter',engine='python', encoding='UTF-8') 
non_suicide_data = pd.read_csv('nao_suicidas.txt',sep='delimiter',engine='python', encoding='UTF-8')

In [4]:
#Tranformação em dataframe
suicide_data = pd.DataFrame(suicide_data)
non_suicide_data = pd.DataFrame(non_suicide_data)

In [5]:
# Atribuir nome à coluna
suicide_data.columns     = ['mensagem'] 
non_suicide_data.columns = ['mensagem'] 

In [ ]:
# Definição das classes: 1 - Mensagens com Ideações Suicidas , 0 - Mensagens sem Ideações Suicidas
suicide_data['label'] = 1
non_suicide_data['label'] = 0
non_suicide_data.head(1)

In [7]:
# Concatenação dos Dados
df = pd.concat([suicide_data,non_suicide_data])

In [8]:
# Seraparação da base de dados para treinamento de teste
train_df, eval_df = train_test_split(df, test_size=0.25)

**AJUSTE FINO**

Nesta etapa serão realizados dois ciclos de ajustes finos. O Ciclo 1 será divido em duas etapas.

A parametrização para ambos os ciclos será considerada a mesma.

In [9]:
# Parâmetros para ajuste fino

train_args={
    'reprocess_input_data': True,
    'overwrite_output_dir': True,
    #'evaluate_during_training': False,
    'logging_steps': 0.5,
    #'stride': 0.8,
    'max_seq_length': 254,
    'num_train_epochs': 3
}

**1.1 ETAPA 1**

Nesta etapa será realizado o ajuste fino com o modelo pré-treinado bert-base-multilingual-cased.


In [ ]:
# BERT MULTILINGUAL

model_Bert = ClassificationModel('bert', 'bert-base-multilingual-cased', args=train_args, use_cuda=False)


In [ ]:
model_Bert.train_model(train_df)

In [ ]:
# Evaluate the model
result, model_outputs, wrong_predictions = model_Bert.eval_model(eval_df, acuracia=acc,precisao = pr,recall = rec, f1 = f1, auc = auc)
result

**1.1 ETAPA 2**

Na Etapa 2 é feita uma nova avaliação do modelo gerado no ajuste fino(etapa 1), porém com novos dados.

In [21]:
eval_model = pd.read_csv('nova_base_avaliacao.csv',sep=';',engine='python', encoding='ISO-8859-1')

In [ ]:
# Evaluate the model
result, model_outputs, wrong_predictions = model_Bert.eval_model(eval_df, acuracia=acc,precisao = pr,recall = rec, f1 = f1, auc = auc)
result

**2° CICLO DE MODELAGEM**

In [25]:
# Leitura dos arquivos de textos com e sem ideação suicida
non_suicide_data = pd.read_csv('nao_suicidas_v2.csv',sep=';',engine='python', encoding='ISO-8859-1')
suicide_data = pd.read_csv('suicidas_v2.csv',sep=';',engine='python', encoding='ISO-8859-1')


In [26]:
#Tranformação em dataframe
suicide_data = pd.DataFrame(suicide_data)
non_suicide_data = pd.DataFrame(non_suicide_data)

In [32]:
# Concatenação dos Dados
df = pd.concat([suicide_data,non_suicide_data])

In [33]:
# Seraparação da base de dados para treinamento de teste
train_df, eval_df = train_test_split(df, test_size=0.25)

In [ ]:
# BERT PORTUGUESE
model_Bert = ClassificationModel('bert', 'neuralmind/bert-base-portuguese-cased', args=train_args, use_cuda=False)

In [ ]:
model_Bert.train_model(train_df)

In [ ]:
# Evaluate the model
result, model_outputs, wrong_predictions = model_Bert.eval_model(eval_df, acuracia=acc,precisao = pr,recall = rec, f1 = f1, auc = auc)
result